In [3]:
import credentials as cd
from fyers_apiv3 import fyersModel
import pandas as pd
import datetime as dt

In [4]:
with open("access.txt", "r") as a:
    access_token = a.read()
client_id = cd.client_id

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(
    client_id=client_id, is_async=False, token=access_token, log_path=""
)

In [ ]:
# data = {
#     "symbol": "NSE:NIFTY50-INDEX",
#     "resolution": "1",
#     "date_format": "1",
#     "range_from": "2024-03-01",
#     "range_to": "2024-05-09",
#     "cont_flag": "1",
# }

# response = fyers.history(data=data)
# print(response)
# data = response["candles"]
# df = pd.DataFrame(data)
# print(df)

# df.columns = ["date", "open", "high", "low", "close", "volume"]
# df["date"] = pd.to_datetime(df["date"], unit="s")

# df.date = df.date.dt.tz_localize("UTC").dt.tz_convert("Asia/Kolkata")
# print(df)
# df["date"] = df["date"].dt.tz_localize(None)
# df = df.set_index("date")

# print(df)
# df.to_csv("data.csv")
# print(dt.datetime.now())

In [ ]:
# def fetchOHLC(ticker, interval, duration):
#     """extracts historical data and outputs in the form of dataframe"""
#     instrument = ticker
#     data = {
#         "symbol": instrument,
#         "resolution": interval,
#         "date_format": "1",
#         "range_from": "2024-03-01",
#         "range_to": "2024-05-09",
#         "cont_flag": "1",
#     }
#     sdata = fyers.history(data)
#     sdata = pd.DataFrame(sdata["candles"])
#     sdata.columns = ["date", "open", "high", "low", "close", "volume"]
#     sdata["date"] = pd.to_datetime(sdata["date"], unit="s")
#     sdata.date = sdata.date.dt.tz_localize("UTC").dt.tz_convert("Asia/Kolkata")
#     sdata["date"] = sdata["date"].dt.tz_localize(None)
#     sdata = sdata.set_index("date")
#     return sdata


# ticker = "NSE:NIFTY50-INDEX"
# data = fetchOHLC(ticker, "1", 60)
# print(data)

In [18]:
def gethistory(symbol1, type, range_from, range_to):
    symbol = "NSE:" + symbol1 + "-" + type
    start = pd.to_datetime(range_from)
    end = pd.to_datetime(range_to)
    sdata = pd.DataFrame()
    while start <= end:
        end2 = start + pd.DateOffset(days=60)
        data = {
            "symbol": symbol,
            "resolution": "1",
            "date_format": "1",
            "range_from": start.strftime("%Y-%m-%d"),
            "range_to": end2.strftime("%Y-%m-%d"),
            "cont_flag": "1",
        }
        s = fyers.history(data)
        s = pd.DataFrame(s["candles"])
        sdata = pd.concat([sdata, s], ignore_index=True)
        start = end2 + dt.timedelta(1)
    sdata.columns = ["date", "open", "high", "low", "close", "volume"]
    sdata["date"] = pd.to_datetime(sdata["date"], unit="s")
    sdata.date = sdata.date.dt.tz_localize("UTC").dt.tz_convert("Asia/Kolkata")
    sdata["date"] = sdata["date"].dt.tz_localize(None)
    sdata = sdata.set_index("date")
    return sdata


data = gethistory("NIFTY50", "INDEX", "2024-03-01", "2024-05-09")
print(data)
data.to_csv("NIFTY50.csv")

                         open      high       low     close  volume
date                                                               
2024-03-01 09:15:00  22048.30  22125.65  22047.75  22117.25       0
2024-03-01 09:16:00  22117.55  22124.05  22117.50  22118.40       0
2024-03-01 09:17:00  22119.45  22122.40  22114.90  22114.90       0
2024-03-01 09:18:00  22113.65  22122.65  22112.35  22121.75       0
2024-03-01 09:19:00  22122.20  22123.40  22114.10  22114.95       0
...                       ...       ...       ...       ...     ...
2024-05-10 14:47:00  22034.35  22036.45  22025.70  22028.90       0
2024-05-10 14:48:00  22028.95  22030.30  22020.60  22024.05       0
2024-05-10 14:49:00  22022.95  22029.55  22021.40  22025.10       0
2024-05-10 14:50:00  22026.45  22034.00  22021.35  22032.45       0
2024-05-10 14:51:00  22033.10  22034.10  22028.45  22030.15       0

[16942 rows x 5 columns]


In [53]:
# # Define the EMA periods
# short_ema_period = 9
# long_ema_period = 13
# signal_line_period = 9

# # Calculate the short-period EMA
# data[f"EMA{short_ema_period}"] = (
#     data["close"].ewm(span=short_ema_period, adjust=False).mean()
# )

# # Calculate the long-period EMA
# data[f"EMA{long_ema_period}"] = (
#     data["close"].ewm(span=long_ema_period, adjust=False).mean()
# )

# # Calculate MACD (the difference between short-period EMA and long-period EMA)
# data["MACD"] = data[f"EMA{short_ema_period}"] - data[f"EMA{long_ema_period}"]

# # Calculate the signal line (the EMA of MACD)
# data["Signal_Line"] = data["MACD"].ewm(span=signal_line_period, adjust=False).mean()

In [ ]:
print(data)

In [55]:
# # Initialize an empty list to store the signals
# signals = []

# # Check for MACD and Signal Line crossovers in the last two rows
# last_row = data.iloc[-1]
# second_last_row = data.iloc[-2]

# if (
#     second_last_row["MACD"] > second_last_row["Signal_Line"]
#     and last_row["MACD"] < last_row["Signal_Line"]
# ):
#     print("Cross Below Signal Line")
#     signals.append(
#         {
#             "timestamp": last_row.name,
#             "price": last_row["close"],
#             "signal": "sell",
#             "MACD": last_row["MACD"],
#             "Signal_Line": last_row["Signal_Line"],
#         }
#     )
# elif (
#     second_last_row["MACD"] < second_last_row["Signal_Line"]
#     and last_row["MACD"] > last_row["Signal_Line"]
# ):
#     print("Cross Above Signal Line")
#     signals.append(
#         {
#             "timestamp": last_row.name,
#             "price": last_row["close"],
#             "signal": "buy",
#             "MACD": last_row["MACD"],
#             "Signal_Line": last_row["Signal_Line"],
#         }
#     )
# else:
#     print("No Crossover")

# # Convert the list of signals to a DataFrame for easier manipulation
# signals_df = pd.DataFrame(signals)
# print(signals_df)

In [ ]:
# import ta


# # Calculate the 9-period and 13-period EMA of the closing prices
# df["EMA9"] = ta.trend.ema_indicator(df.close, window=9)
# df["EMA13"] = ta.trend.ema_indicator(df.close, window=13)

# # Calculate the MACD with fast period of 9, slow period of 13, and signal period of 9
# macd = ta.trend.MACD(df.close, window_fast=9, window_slow=13, window_sign=9)
# df["MACD"], df["MACD_signal"], df["MACD_hist"] = (
#     macd.macd(),
#     macd.macd_signal(),
#     macd.macd_diff(),
# )

# # Initialize an empty list to store the signals
# signals = []

# # Loop over all rows in the data (starting from the second one)
# for i in range(
#     1, len(df) - 1
# ):  # Adjusted to len(df)-1 to avoid index error in the next line
#     # Get the current row, the previous row, and the next row
#     current_row = df.iloc[i]
#     previous_row = df.iloc[i - 1]
#     next_row = df.iloc[i + 1]  # Added next_row

#     # Identify the trend based on the MACD and Signal Line
#     if current_row["MACD"] > current_row["MACD_signal"]:
#         trend = "up"
#     else:
#         trend = "down"

#     # Check for EMA crossovers
#     if (
#         previous_row["EMA9"] > previous_row["EMA13"]
#         and current_row["EMA9"] < current_row["EMA13"]
#         and trend == "down"
#     ):
#         signals.append(
#             {
#                 "timestamp": current_row.name,
#                 "price": current_row["close"],
#                 "signal": "sell",
#                 "MACD": current_row["MACD"],
#                 "Signal_Line": current_row["MACD_signal"],
#                 "trend": trend,
#             }
#         )
#     elif (
#         previous_row["EMA9"] < previous_row["EMA13"]
#         and current_row["EMA9"] > current_row["EMA13"]
#         and trend == "up"
#     ):
#         signals.append(
#             {
#                 "timestamp": current_row.name,
#                 "price": current_row["close"],
#                 "signal": "buy",
#                 "MACD": current_row["MACD"],
#                 "Signal_Line": current_row["MACD_signal"],
#                 "trend": trend,
#             }
#         )

# # Convert the list of signals to a DataFrame for easier manipulation
# signals_df = pd.DataFrame(signals)

# # Calculate profit/loss
# signals_df["profit_loss"] = signals_df["price"].diff()
# # For 'buy' signals, we need to take the negative of the profit/loss
# signals_df.loc[signals_df["signal"] == "buy", "profit_loss"] *= -1

# # Output the DataFrame to a CSV file
# signals_df.to_csv("signals.csv", index=False)

# print(signals_df)

In [20]:
data = {"symbol": "NSE:NIFTY50-INDEX", "strikecount": 1, "timestamp": ""}
response = fyers.optionchain(data=data)
print(response)



{'code': 200, 'data': {'callOi': 88016450, 'expiryData': [{'date': '16-05-2024', 'expiry': '1715853600'}, {'date': '23-05-2024', 'expiry': '1716458400'}, {'date': '30-05-2024', 'expiry': '1717063200'}, {'date': '06-06-2024', 'expiry': '1717668000'}, {'date': '13-06-2024', 'expiry': '1718272800'}, {'date': '27-06-2024', 'expiry': '1719482400'}, {'date': '25-07-2024', 'expiry': '1721901600'}, {'date': '26-09-2024', 'expiry': '1727344800'}, {'date': '26-12-2024', 'expiry': '1735207200'}, {'date': '27-03-2025', 'expiry': '1743069600'}, {'date': '26-06-2025', 'expiry': '1750932000'}, {'date': '24-12-2025', 'expiry': '1766570400'}, {'date': '25-06-2026', 'expiry': '1782381600'}, {'date': '31-12-2026', 'expiry': '1798711200'}, {'date': '24-06-2027', 'expiry': '1813831200'}, {'date': '30-12-2027', 'expiry': '1830160800'}, {'date': '29-06-2028', 'expiry': '1845885600'}, {'date': '28-12-2028', 'expiry': '1861610400'}], 'indiavixData': {'ask': 0, 'bid': 0, 'description': 'INDIAVIX-INDEX', 'ex_sym

In [19]:
import ta


# Calculate the 9-period and 13-period EMA of the closing prices
df["EMA9"] = ta.trend.ema_indicator(df.close, window=9)
df["EMA13"] = ta.trend.ema_indicator(df.close, window=13)

# Calculate the MACD with fast period of 9, slow period of 13, and signal period of 9
macd = ta.trend.MACD(df.close, window_fast=9, window_slow=13, window_sign=9)
df["MACD"], df["MACD_signal"], df["MACD_hist"] = (
    macd.macd(),
    macd.macd_signal(),
    macd.macd_diff(),
)

# Initialize an empty list to store the signals
signals = []

# Loop over all rows in the data (starting from the second one)
for i in range(
    1, len(df) - 1
):  # Adjusted to len(df)-1 to avoid index error in the next line
    # Get the current row, the previous row, and the next row
    current_row = df.iloc[i]
    previous_row = df.iloc[i - 1]
    next_row = df.iloc[i + 1]  # Added next_row

    # Identify the trend based on the MACD and Signal Line
    if current_row["MACD"] > current_row["MACD_signal"]:
        trend = "up"
    else:
        trend = "down"

    # Check for EMA crossovers
    if (
        previous_row["EMA9"] > previous_row["EMA13"]
        and current_row["EMA9"] < current_row["EMA13"]
        and trend == "down"
    ):
        signals.append(
            {
                "timestamp": current_row.name,
                "price": current_row["close"],
                "signal": "sell",
                "MACD": current_row["MACD"],
                "Signal_Line": current_row["MACD_signal"],
                "trend": trend,
            }
        )
    elif (
        previous_row["EMA9"] < previous_row["EMA13"]
        and current_row["EMA9"] > current_row["EMA13"]
        and trend == "up"
    ):
        signals.append(
            {
                "timestamp": current_row.name,
                "price": current_row["close"],
                "signal": "buy",
                "MACD": current_row["MACD"],
                "Signal_Line": current_row["MACD_signal"],
                "trend": trend,
            }
        )

# Convert the list of signals to a DataFrame for easier manipulation
signals_df = pd.DataFrame(signals)

# Calculate profit/loss
signals_df["profit_loss"] = signals_df["price"].diff()
# For 'buy' signals, we need to take the negative of the profit/loss
signals_df.loc[signals_df["signal"] == "buy", "profit_loss"] *= -1

# Output the DataFrame to a CSV file
signals_df.to_csv("signals.csv", index=False)

print(signals_df)

              timestamp     price signal      MACD  Signal_Line trend  \
0   2024-03-01 10:37:00  22193.50   sell -0.038542     0.890677  down   
1   2024-03-01 10:53:00  22192.40    buy  0.158421    -0.687299    up   
2   2024-03-01 10:59:00  22172.45   sell -0.671781     0.139517  down   
3   2024-03-01 11:09:00  22191.20    buy  0.147055    -0.358836    up   
4   2024-03-01 12:03:00  22275.85   sell -0.211512     1.422534  down   
..                  ...       ...    ...       ...          ...   ...   
873 2024-05-09 14:08:00  22060.30    buy  0.794929    -0.773288    up   
874 2024-05-09 14:19:00  22047.95   sell -0.054060     0.686606  down   
875 2024-05-09 14:28:00  22042.85   sell -0.730840     0.348097  down   
876 2024-05-09 14:54:00  22008.65    buy  0.048029    -2.491871    up   
877 2024-05-09 15:28:00  21962.05    buy  0.068718    -2.112942    up   

     profit_loss  
0            NaN  
1           1.10  
2         -19.95  
3         -18.75  
4          84.65  
..       